# Laporan Proyek UAS: Klasifikasi Time Series Audio (Anjing vs Kucing)

**Nama:** Hajjid Rafi Mumtaz  
**NIM:** 230411100078  
**Dataset Source:** [Kaggle - Audio Cats and Dogs](https://www.kaggle.com/datasets/mmoreaux/audio-cats-and-dogs)

---

## 1. Business Understanding (Memahami Bisnis)

Tahap ini berfokus pada pemahaman tujuan proyek dan kebutuhan analisis dari sudut pandang pemecahan masalah.

### 1.1 Latar Belakang Masalah
Data audio merupakan salah satu bentuk data *time series* (deret waktu) di mana sinyal suara berubah seiring berjalannya waktu (amplitudo vs waktu). Dalam konteks pemrosesan sinyal digital, membedakan jenis suara hewan secara otomatis memiliki tantangan tersendiri karena variasi frekuensi, durasi, dan *noise* latar belakang.

### 1.2 Tujuan Proyek (Project Objectives)
Tujuan utama dari proyek ini adalah membangun model *Machine Learning* (atau *Deep Learning*) yang mampu melakukan klasifikasi biner pada data *time series* audio untuk:
1.  **Mendeteksi Pola Suara:** Mempelajari fitur karakteristik dari gelombang suara anjing (gonggongan) dan kucing (meong).
2.  **Otomasi Klasifikasi:** Mengklasifikasikan file audio baru secara akurat ke dalam label "Dog" atau "Cat".

### 1.3 Tujuan Analisis
* Mengekstrak fitur bermakna dari data mentah audio (seperti MFCC atau Spektrogram).
* Membangun model klasifikasi yang memiliki akurasi, presisi, dan recall yang baik.
* Menyediakan model yang siap di-*deploy* ke aplikasi web (Streamlit).

## 2. Data Understanding (Memahami Data / EDA)

Tahap ini bertujuan untuk mengumpulkan data, mendeskripsikan data, serta melakukan eksplorasi data (Exploratory Data Analysis/EDA) untuk mengenali karakteristik sinyal audio.


### 2.1 Pengumpulan Data
Dataset diambil dari Kaggle dengan judul "Audio Cats and Dogs". Dataset ini berisi file audio dalam format `.wav`.
* **Sumber:** Kaggle (mmoreaux/audio-cats-and-dogs)
* **Struktur Folder:** Terbagi menjadi folder `cats_dogs` yang berisi subfolder label atau file tercampur dengan penamaan khusus.

### 2.2 Deskripsi Data
Pada tahap ini, dilakukan pemeriksaan properti fisik dari file audio:
* **Jumlah Sampel:** Menghitung total file audio untuk kucing dan anjing.
* **Durasi Audio:** Memeriksa rata-rata panjang durasi audio (detik).
* **Sample Rate:** Memeriksa frekuensi pengambilan sampel (misalnya 16kHz atau 44.1kHz).
* **Channel:** Memastikan apakah audio mono atau stereo.

### 2.3 Eksplorasi Data (EDA)
Visualisasi dilakukan untuk memahami perbedaan pola *time series* antara kedua kelas:

1.  **Visualisasi Waveform (Time Domain):**
    * Plot Amplitudo vs Waktu.
    * *Observasi:* Melihat bentuk gelombang mentah, apakah suara anjing memiliki amplitudo yang lebih meledak-ledak dibandingkan kucing.
2.  **Visualisasi Spektrogram (Frequency Domain):**
    * Mengubah sinyal waktu menjadi representasi visual frekuensi.
    * *Observasi:* Melihat intensitas frekuensi pada waktu tertentu. Gonggongan anjing biasanya memiliki frekuensi rendah yang kuat, sedangkan kucing mungkin memiliki harmonik di frekuensi lebih tinggi.
3.  **Analisis Keseimbangan Kelas (Class Balance):**
    * Memastikan jumlah data *Cat* dan *Dog* seimbang agar model tidak bias.

## 3. Preprocessing Data

Tahap ini adalah proses mengubah data audio mentah menjadi format yang dapat dipahami oleh algoritma *machine learning*. Karena ini adalah data *time series*, fitur temporal sangat penting.

### 3.1 Data Cleaning & Loading
* Menggunakan *library* `librosa` untuk memuat file audio.
* Menghapus file yang korup atau memiliki durasi terlalu pendek (misal < 1 detik) yang tidak mengandung informasi cukup.

### 3.2 Resampling
* Menyamakan *sample rate* seluruh file audio (misalnya diseragamkan menjadi 22.050 Hz) untuk menjaga konsistensi dimensi data.

### 3.3 Fixing Duration (Padding & Truncating)
Model klasifikasi (terutama Neural Network seperti CNN atau LSTM) membutuhkan input dengan dimensi yang tetap (fixed shape).
* **Truncating:** Jika audio lebih panjang dari durasi target (misal 5 detik), audio akan dipotong.
* **Padding:** Jika audio lebih pendek dari target, akan ditambahkan nilai nol (*zero padding*) hingga mencapai panjang yang ditentukan.

### 3.4 Feature Extraction (Ekstraksi Fitur)
Mengubah data *time series* mentah menjadi fitur numerik yang lebih padat informasi. Metode yang digunakan:
* **Mel-Frequency Cepstral Coefficients (MFCCs):** Teknik ini sangat populer dalam pengolahan suara karena merepresentasikan amplop spektral pendek dari gelombang suara yang menyerupai cara telinga manusia mendengar.
    * *Output:* Matriks 2D (koefisien MFCC x waktu).

### 3.5 Label Encoding
* Mengubah label kategori menjadi numerik:
    * `Cat` -> 0
    * `Dog` -> 1

### 3.6 Train-Test Split
* Membagi dataset menjadi:
    * **Training Set (80%):** Untuk melatih model.
    * **Test Set (20%):** Untuk evaluasi performa model.

## 4. Modeling (Pemodelan Data)

Tahap ini adalah inti dari proses *machine learning* di mana algoritma diaplikasikan untuk mempelajari pola dari fitur audio yang telah diekstraksi.

### 4.1 Pemilihan Algoritma
Mengingat data berupa *time series* audio yang telah diekstraksi fiturnya (MFCC), model yang dipilih adalah **Artificial Neural Network (ANN)** atau **Convolutional Neural Network (1D-CNN)**.
* **Alasan:** Neural Network sangat baik dalam menangkap pola non-linear yang kompleks pada data spektrogram/MFCC audio dibandingkan algoritma klasik.

### 4.2 Desain Arsitektur Model
* **Input Layer:** Menerima array fitur (jumlah koefisien MFCC).
* **Hidden Layers:** Menggunakan lapisan *Dense* (Fully Connected) dengan fungsi aktivasi **ReLU** untuk mempelajari representasi fitur.
* **Output Layer:** Satu neuron dengan fungsi aktivasi **Sigmoid** (karena ini klasifikasi biner: 0 atau 1 / Kucing atau Anjing).

### 4.3 Konfigurasi Pelatihan (Training)
* **Loss Function:** `Binary Crossentropy` (standar untuk klasifikasi biner).
* **Optimizer:** `Adam` (adaptif dan efisien).
* **Metric:** `Accuracy`.
* **Epochs:** Jumlah iterasi pelatihan (misal: 50-100 epoch).

## 5. Evaluation (Evaluasi)

Setelah model dilatih, performanya harus divalidasi menggunakan data uji (*Test Set*) yang tidak pernah dilihat model sebelumnya.

### 5.1 Metrik Evaluasi
* **Akurasi (Accuracy):** Seberapa sering model menebak dengan benar secara keseluruhan.
* **Confusion Matrix:** Melihat detail kesalahan prediksi (berapa banyak suara kucing diprediksi sebagai anjing, dan sebaliknya).
* **Classification Report:** Menampilkan *Precision*, *Recall*, dan *F1-Score*.

### 5.2 Analisis Hasil
* Apakah model mengalami *Overfitting* (bagus di training, buruk di testing)?
* Apakah model bias ke salah satu kelas (misal: jago menebak anjing tapi gagal menebak kucing)?

## 6. Deployment

Tahap terakhir adalah membuat model dapat digunakan oleh pengguna akhir melalui antarmuka grafis.

### 6.1 Rencana Deployment
* **Platform:** Streamlit (Framework Python untuk Web Apps Data Science).
* **Fitur:**
    * Upload file audio (`.wav`).
    * Visualisasi audio (Audio player & Waveform).
    * Tombol "Prediksi".
    * Output hasil klasifikasi (Kucing/Anjing) beserta tingkat keyakinan (confidence score).